# Ensamblar y calcular ganancia

### Autor: Federico Picado
### Fecha de última modificación: 28/10/2024
### Descripción:

Podemos evaluar la ganancia de un modelo en particular, ordenado de distinta forma.
Tambien podemos realizar ensambles entre modelos y evaluar su ganancia. 

## Parámetros

In [1]:
#ver

## Input

In [12]:
# Ya en el input se ejecutan cosas
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import time

In [11]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'# Cambiar
dataset_path = base_path + 'datasets/'
dataset_file = 'Competencia_01_tagg.csv' # df con N_cliente, clase_ternaria para junio
clientes_baja_junio= pd.read_csv(dataset_path + dataset_file)

# nombre de la carpeta donde se guardaron los resultados para cada modelo segun la metrica
dataset_folder1="prediccion_gv"
dataset_folder2="prediccion_shap"
#en el caso de tener mas modelos agregarlos


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [98]:
# en este ultimo jobs se evalua la performance del modelo.

# MAIN

In [4]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Función para cargar los CSV de una carpeta específica y asignarlos a variables
def cargar_csv_de_carpeta(dataset_folder):
    folder_path = os.path.join(dataset_path, dataset_folder)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            # Nombre de la variable sin extensión
            variable_name = file_name.replace('.csv', '')
            
            # Cargar el DataFrame
            file_path = os.path.join(folder_path, file_name)
            dataframes[variable_name] = pd.read_csv(file_path)
            print(f"Archivo '{file_name}' cargado en la variable '{variable_name}'")

# Cargar los archivos de cada carpeta
cargar_csv_de_carpeta(dataset_folder1)
cargar_csv_de_carpeta(dataset_folder2)


Archivo 'prediccion_gv_bootstrap.csv' cargado en la variable 'prediccion_gv_bootstrap'
Archivo 'prediccion_gv_max.csv' cargado en la variable 'prediccion_gv_max'
Archivo 'prediccion_gv_mediana.csv' cargado en la variable 'prediccion_gv_mediana'
Archivo 'prediccion_gv_promedio.csv' cargado en la variable 'prediccion_gv_promedio'
Archivo 'prediccion_shap_bootstrap.csv' cargado en la variable 'prediccion_shap_bootstrap'
Archivo 'prediccion_shap_max.csv' cargado en la variable 'prediccion_shap_max'
Archivo 'prediccion_shap_mediana.csv' cargado en la variable 'prediccion_shap_mediana'
Archivo 'prediccion_shap_promedio.csv' cargado en la variable 'prediccion_shap_promedio'


In [8]:
dataframes

{'prediccion_gv_bootstrap':         numero_de_cliente  Probabilidad
 0              1462097870      0.912031
 1               606371992      0.898902
 2               560453687      0.897263
 3               627202522      0.878225
 4              1455714205      0.873836
 ...                   ...           ...
 164871          576296876      0.000009
 164872          566164387      0.000008
 164873          590935893      0.000007
 164874          652025650      0.000007
 164875          577768545      0.000007
 
 [164876 rows x 2 columns],
 'prediccion_gv_max':         numero_de_cliente  Probabilidad
 0               606371992      0.937074
 1              1462097870      0.933110
 2               627202522      0.920704
 3               560453687      0.914898
 4              1455714205      0.901547
 ...                   ...           ...
 164871          576296876      0.000016
 164872          716625469      0.000016
 164873          590935893      0.000014
 164874          652

In [26]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=123
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

In [27]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)

# Iteramos sobre cada modelo en el diccionario de predicciones
for model_name, pred_model in dataframes.items():
    # Aseguramos que las predicciones estén ordenadas por probabilidad descendente
    pred_model_sorted = pred_model.sort_values('Probabilidad', ascending=False)
    
    # Iteramos sobre cada corte
    for corte in cortes:
        ganancia_publico, ganancia_privado = calculoGanancia(clientes_baja_junio, pred_model_sorted, corte)
        
        # Almacenamos los resultados
        resultados.append({
            'Modelo': model_name,
            'Corte': corte,
            'Ganancia Público': ganancia_publico,
            'Ganancia Privado': ganancia_privado
        })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")


Tiempo de ejecución: 14.97 segundos


In [28]:
resultados

,Modelo,Corte,Ganancia Público,Ganancia Privado
0,prediccion_gv_bootstrap,5000,7.868000e+07,74680000.0
1,prediccion_gv_bootstrap,6000,8.507333e+07,84340000.0
2,prediccion_gv_bootstrap,7000,8.831667e+07,88550000.0
3,prediccion_gv_bootstrap,8000,8.878333e+07,94350000.0
4,prediccion_gv_bootstrap,9000,9.079000e+07,95890000.0
...,...,...,...,...
115,prediccion_shap_promedio,15000,8.950667e+07,93640000.0
116,prediccion_shap_promedio,16000,8.694000e+07,93140000.0
117,prediccion_shap_promedio,17000,8.575000e+07,88850000.0
118,prediccion_shap_promedio,18000,8.414000e+07,85140000.0


In [29]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_prediccion_gv_bootstrap,Ganancia Privado_prediccion_gv_bootstrap,Ganancia Público_prediccion_gv_max,Ganancia Privado_prediccion_gv_max,Ganancia Público_prediccion_gv_mediana,Ganancia Privado_prediccion_gv_mediana,Ganancia Público_prediccion_gv_promedio,Ganancia Privado_prediccion_gv_promedio,Ganancia Público_prediccion_shap_bootstrap,Ganancia Privado_prediccion_shap_bootstrap,Ganancia Público_prediccion_shap_max,Ganancia Privado_prediccion_shap_max,Ganancia Público_prediccion_shap_mediana,Ganancia Privado_prediccion_shap_mediana,Ganancia Público_prediccion_shap_promedio,Ganancia Privado_prediccion_shap_promedio
0,5000,7.868000e+07,74680000.0,7.310333e+07,72670000.0,7.861000e+07,75110000.0,7.858667e+07,74720000.0,7.329000e+07,74190000.0,7.170333e+07,72470000.0,7.424667e+07,74580000.0,7.408333e+07,73450000.0
1,6000,8.507333e+07,84340000.0,8.078000e+07,82580000.0,8.411667e+07,83950000.0,8.411667e+07,83950000.0,8.402333e+07,82390000.0,8.621667e+07,78250000.0,8.332333e+07,83890000.0,8.409333e+07,82760000.0
2,7000,8.831667e+07,88550000.0,8.670667e+07,88840000.0,8.561000e+07,89310000.0,8.827000e+07,88570000.0,8.981000e+07,89510000.0,8.894667e+07,87880000.0,8.794333e+07,89110000.0,8.981000e+07,89110000.0
3,8000,8.878333e+07,94350000.0,8.964667e+07,91580000.0,8.908667e+07,95420000.0,8.885333e+07,95120000.0,8.955333e+07,92820000.0,9.230667e+07,90440000.0,9.034667e+07,93280000.0,8.955333e+07,92820000.0
4,9000,9.079000e+07,95890000.0,8.743000e+07,93330000.0,9.177000e+07,95470000.0,9.083667e+07,95870000.0,9.198000e+07,96580000.0,9.104667e+07,96580000.0,9.002000e+07,97420000.0,9.207333e+07,96540000.0
5,10000,8.747667e+07,98510000.0,8.803667e+07,94670000.0,8.827000e+07,98970000.0,8.743000e+07,98530000.0,9.331000e+07,96810000.0,8.873667e+07,97570000.0,9.326333e+07,96430000.0,9.328667e+07,96820000.0
6,11000,8.635667e+07,97390000.0,8.645000e+07,97350000.0,8.722000e+07,97420000.0,8.729000e+07,96990000.0,9.380000e+07,99400000.0,9.023000e+07,95330000.0,9.375333e+07,98220000.0,9.466333e+07,98630000.0
7,12000,8.927333e+07,98140000.0,9.020667e+07,94540000.0,8.852667e+07,97260000.0,8.927333e+07,98540000.0,9.011333e+07,98180000.0,9.053333e+07,95600000.0,9.118667e+07,98520000.0,9.020667e+07,98540000.0
8,13000,8.855000e+07,94850000.0,8.556333e+07,94530000.0,8.850333e+07,94870000.0,8.852667e+07,95260000.0,9.086000e+07,96260000.0,9.086000e+07,96260000.0,8.990333e+07,96270000.0,9.170000e+07,96300000.0
9,14000,8.680000e+07,92000000.0,8.530667e+07,91840000.0,8.948333e+07,91650000.0,8.696333e+07,91930000.0,8.911000e+07,95010000.0,9.121000e+07,94110000.0,8.750000e+07,95700000.0,8.747667e+07,94110000.0


## Tengo el problema que segun la semilla con la que dividi el publico privado me puede dar distinto.. hago semillero

In [32]:
def calculoGanancia(bajas, prediccion, corte, random_state):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia normalizada para el conjunto público.
    ganancia_privado: Ganancia normalizada para el conjunto privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=random_state
    )

    # Obtener los estímulos según el modelo
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # Calcular la ganancia para cada conjunto con normalización
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

def calculoGananciaPromedio(bajas, prediccion, corte, n_seeds=100):
    '''
    Calcula la ganancia promedio para múltiples semillas.
    
    Parámetros:
    bajas: Df con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: Df con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    n_seeds: int, número de semillas aleatorias (por defecto 200).
    
    Retorna:
    promedio_ganancia_publico: Ganancia promedio normalizada para el conjunto público.
    promedio_ganancia_privado: Ganancia promedio normalizada para el conjunto privado.
    '''
    
    seeds = np.random.randint(1000, 100001, size=n_seeds)
    ganancias_publico = []
    ganancias_privado = []

    for seed in seeds:
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, prediccion, corte, random_state=seed)
        ganancias_publico.append(ganancia_publico)
        ganancias_privado.append(ganancia_privado)

    promedio_ganancia_publico = np.mean(ganancias_publico)
    promedio_ganancia_privado = np.mean(ganancias_privado)

    return promedio_ganancia_publico,promedio_ganancia_privado


In [33]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)

# Iteramos sobre cada modelo en el diccionario de predicciones
for model_name, pred_model in dataframes.items():
    # Aseguramos que las predicciones estén ordenadas por probabilidad descendente
    pred_model_sorted = pred_model.sort_values('Probabilidad', ascending=False)
    
    # Iteramos sobre cada corte
    for corte in cortes:
        ganancia_publico, ganancia_privado = calculoGananciaPromedio(clientes_baja_junio, pred_model_sorted, corte)
        
        # Almacenamos los resultados
        resultados.append({
            'Modelo': model_name,
            'Corte': corte,
            'Ganancia Público': ganancia_publico,
            'Ganancia Privado': ganancia_privado
        })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 1486.60 segundos


In [34]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_prediccion_gv_bootstrap,Ganancia Privado_prediccion_gv_bootstrap,Ganancia Público_prediccion_gv_max,Ganancia Privado_prediccion_gv_max,Ganancia Público_prediccion_gv_mediana,Ganancia Privado_prediccion_gv_mediana,Ganancia Público_prediccion_gv_promedio,Ganancia Privado_prediccion_gv_promedio,Ganancia Público_prediccion_shap_bootstrap,Ganancia Privado_prediccion_shap_bootstrap,Ganancia Público_prediccion_shap_max,Ganancia Privado_prediccion_shap_max,Ganancia Público_prediccion_shap_mediana,Ganancia Privado_prediccion_shap_mediana,Ganancia Público_prediccion_shap_promedio,Ganancia Privado_prediccion_shap_promedio
0,5000,7.527333e+07,76140000.0,7.326923e+07,72598900.0,7.637350e+07,76068500.0,7.631563e+07,75693300.0,7.318197e+07,74236300.0,7.276407e+07,72015400.0,7.384440e+07,74752400.0,7.400073e+07,73485400.0
1,6000,8.436540e+07,84643400.0,8.229620e+07,81930200.0,8.365233e+07,84149000.0,8.484723e+07,83636900.0,8.273393e+07,82942600.0,8.003193e+07,80900600.0,8.339427e+07,83859600.0,8.301627e+07,83221600.0
2,7000,8.819463e+07,88602300.0,8.881087e+07,87938200.0,8.721860e+07,88620600.0,8.910813e+07,88210800.0,8.806000e+07,90260000.0,8.769460e+07,88416600.0,8.866363e+07,88801300.0,8.873783e+07,89569500.0
3,8000,9.243640e+07,92784400.0,9.015673e+07,91361400.0,9.356830e+07,93499300.0,9.255960e+07,93531600.0,9.253697e+07,91541300.0,9.095287e+07,91020200.0,9.286737e+07,92199700.0,9.386020e+07,90974200.0
4,9000,9.373257e+07,94628900.0,9.026337e+07,92115700.0,9.377247e+07,94611800.0,9.486143e+07,94145100.0,9.606497e+07,94829300.0,9.515987e+07,94817200.0,9.530430e+07,95155300.0,9.556493e+07,95043600.0
5,10000,9.519020e+07,95204200.0,9.307993e+07,92508600.0,9.467080e+07,96226800.0,9.402913e+07,95701800.0,9.445777e+07,96318100.0,9.363480e+07,95470800.0,9.552153e+07,95462200.0,9.642873e+07,95473400.0
6,11000,9.413833e+07,94055000.0,9.322273e+07,94447400.0,9.475293e+07,94191600.0,9.443397e+07,93928300.0,9.691897e+07,98063300.0,9.442510e+07,93532100.0,9.653490e+07,97027900.0,9.732450e+07,97489500.0
7,12000,9.573830e+07,95369300.0,9.301157e+07,93337900.0,9.523920e+07,94383200.0,9.476203e+07,96187700.0,9.588763e+07,95705300.0,9.527630e+07,93567300.0,9.773587e+07,95713200.0,9.501823e+07,96477900.0
8,13000,9.301600e+07,92936000.0,9.218533e+07,91692000.0,9.366700e+07,92657000.0,9.409283e+07,92874500.0,9.317770e+07,95266700.0,9.453547e+07,94684800.0,9.452590e+07,94288900.0,9.523920e+07,94783200.0
9,14000,9.001883e+07,90620500.0,8.953047e+07,90029800.0,8.992200e+07,91462000.0,8.907710e+07,91024100.0,9.225417e+07,93662500.0,9.329273e+07,93217400.0,9.383967e+07,92983000.0,9.206213e+07,92144800.0


In [37]:
resultados_pivot.to_csv("Resumen_predicciones.csv",index=False)